In [1]:
import pandas as pd
from sqlalchemy import create_engine
# from config import username, password
import json
import requests 
from pprint import pprint
import time
import pandas as pd
from sqlalchemy import create_engine 
from flask import jsonify

In [2]:
rds_connection_string = f"postgres:4lll0v3@localhost:5432/park_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
engine.table_names()

<ipython-input-3-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['park',
 'webcam_url',
 'images',
 'fire',
 'sci_name',
 'park_species',
 'activity',
 'park_activities',
 'park_fee',
 'fees']

In [4]:
#Query 'park' table as df
park = pd.read_sql_query('select * from park', con=engine)
park

,park_id,park_name,park_url,park_code,description,latitude,longitude,states,directions_info,directions_url,weather_info,designation
0,77E0D7F0-1942-494A-ACE2-9004D2BDC59E,Abraham Lincoln Birthplace National Historical...,https://www.nps.gov/abli/index.htm,abli,For over a century people from around the worl...,37.585866,-85.673305,KY,The Birthplace Unit of the park is located app...,http://www.nps.gov/abli/planyourvisit/directio...,There are four distinct seasons in Central Ken...,National Historical Park
1,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Acadia National Park,https://www.nps.gov/acad/index.htm,acad,Acadia National Park protects the natural beau...,44.409286,-68.247501,ME,"From Boston take I-95 north to Augusta, Maine,...",http://www.nps.gov/acad/planyourvisit/directio...,"Located on Mount Desert Island in Maine, Acadi...",National Park
2,E4C7784E-66A0-4D44-87D0-3E072F5FEF43,Adams National Historical Park,https://www.nps.gov/adam/index.htm,adam,From the sweet little farm at the foot of Penn...,42.255396,-71.011604,MA,"Traveling on U.S. Interstate 93, take exit 7 -...",http://www.nps.gov/adam/planyourvisit/directio...,"Be prepared for hot, humid weather. The histor...",National Historical Park
3,1A47416F-DAA3-4137-9F30-14AF86B4E547,African American Civil War Memorial,https://www.nps.gov/afam/index.htm,afam,"Over 200,000 African-American soldiers and sai...",38.916600,-77.026000,DC,The memorial is located at the corner of Vermo...,http://www.nps.gov/afam/planyourvisit/directio...,Washington DC gets to see all four seasons. Hu...,Not Available
4,E6E1D22A-7A89-47F8-813C-B611059A8CF9,African Burial Ground National Monument,https://www.nps.gov/afbg/index.htm,afbg,African Burial Ground is the oldest and larges...,40.714527,-74.004474,NY,The African Burial Ground National Monument is...,http://www.nps.gov/afbg/planyourvisit/directio...,http://forecast.weather.gov/MapClick.php?CityN...,National Monument
...,...,...,...,...,...,...,...,...,...,...,...,...
453,F58C6D24-8D10-4573-9826-65D42B8B83AD,Yellowstone National Park,https://www.nps.gov/yell/index.htm,yell,"On March 1, 1872, Yellowstone became the first...",44.598244,-110.547169,"ID,MT,WY","Yellowstone National Park covers nearly 3,500 ...",http://www.nps.gov/yell/planyourvisit/directio...,"Yellowstone's weather can vary quite a bit, ev...",National Park
454,D09FF0F8-B16E-4D2B-9C37-03202D6B8176,Yorktown Battlefield Part of Colonial National...,https://www.nps.gov/york/index.htm,york,Discover what it took for the United States to...,37.219500,-76.498300,VA,"For an internet map search or GPS, use the fol...",http://www.nps.gov/york/planyourvisit/directio...,"Yorktown, VA climate is warm during summer whe...",Part of Colonial National Historical Park
455,4324B2B4-D1A3-497F-8E6B-27171FAE4DB2,Yosemite National Park,https://www.nps.gov/yose/index.htm,yose,"Not just a great valley, but a shrine to human...",37.848833,-119.557187,CA,You can drive to Yosemite year-round and enter...,http://www.nps.gov/yose/planyourvisit/driving.htm,"Yosemite National Park covers nearly 1,200 squ...",National Park
456,9854D136-AFC0-4966-BB40-FE9323B56A49,Yucca House National Monument,https://www.nps.gov/yuho/index.htm,yuho,Through a continuing tradition of public and p...,37.247789,-108.686127,CO,"•From Cortez, take Hwy. 491 south approximatel...",http://www.nps.gov/yuho/planyourvisit/directio...,Spring and Fall are mild with daytime temperat...,National Monument


In [5]:
#Query 'fire' table as df, 
fire = pd.read_sql_query('select * from fire', con=engine)
park_fire_count = fire.groupby(['park_id']).count()
park_fire_count['fire_count'] = park_fire_count[['fire_id']]
fire_count_df = park_fire_count[['fire_count']]
fire_count_df.head()

,fire_count
park_id,
00E65872-53F7-4C9B-BD29-B9BC99930D0D,1
0103833B-8114-4483-9754-11B5171BCB20,17
021F297B-D672-4794-8C5E-BB58743F252A,12
0265E5A4-576A-4626-882A-05971DEE9B76,1
061D9754-B947-48FD-99BE-1FE1AAD1BB90,536


In [6]:
species = pd.read_sql_query('select * from park_species', con=engine)
species = species.groupby(['park_id','category_name']).count()
species['species_count'] = species[['sci_name_id']]
species = species[['species_count']] 
species = species.reset_index()
species = species.pivot(index='park_id',
                        columns='category_name',
                        values='species_count')
species.head()

category_name,Amphibian,Bird,Mammal,Reptile
park_id,,,,
041B325C-A34F-4027-8E41-1DF3F9A1D799,19.0,287.0,85.0,24.0
07229CB8-8533-4669-B614-2B884779DD93,14.0,148.0,57.0,5.0
0F6893CF-FC15-4AC5-8C95-E70FC9C21B1A,8.0,243.0,18.0,3.0
11E73438-0CCC-4441-A76A-1995F67F2D89,NaN,143.0,30.0,NaN
167A05D1-5793-49E0-89FE-0A1DDFA9A7F4,NaN,280.0,6.0,1.0


In [7]:
images = pd.read_sql_query('select * from images', con=engine)
images = images[['park_id','image_url']]
park_list = images['park_id'].unique().tolist()
park_list_images_df = pd.DataFrame(images.groupby('park_id')['image_url'].apply(list))
park_list_images_df.head()

,image_url
park_id,
00E65872-53F7-4C9B-BD29-B9BC99930D0D,[https://www.nps.gov/common/uploads/structured...
0103833B-8114-4483-9754-11B5171BCB20,[https://www.nps.gov/common/uploads/structured...
01DE3F7C-354D-4468-AC36-2CF5274AB68E,[https://www.nps.gov/common/uploads/structured...
021F297B-D672-4794-8C5E-BB58743F252A,[https://www.nps.gov/common/uploads/structured...
0260D050-761A-4446-AB05-641986F9FDFB,[https://www.nps.gov/common/uploads/structured...


In [8]:
webcam_url = pd.read_sql_query('select * from webcam_url', con=engine)
webcam_url_df = pd.DataFrame(webcam_url.groupby('park_id').agg({'webcam_title': list,
                                                                'park_webcam_url': list}))
webcam_url_df

,webcam_title,park_webcam_url
park_id,,
05F2A0A7-317D-4C4F-95D6-F046BE96310C,[Nez Perce National Historical Park - Big Hole...,[https://www.nps.gov/media/webcam/view.htm?id=...
061D9754-B947-48FD-99BE-1FE1AAD1BB90,"[Split Mountain, Current Air Quality]",[https://www.nps.gov/media/webcam/view.htm?id=...
07229CB8-8533-4669-B614-2B884779DD93,"[Air Quality, Sunrise Mountain, Mountain, West...",[https://www.nps.gov/media/webcam/view.htm?id=...
0F6893CF-FC15-4AC5-8C95-E70FC9C21B1A,"[Rock Harbor Webcam, Windigo Webcam, Mott Isla...",[https://www.nps.gov/media/webcam/view.htm?id=...
0FC25325-87AA-4325-A6B7-C404D94B16DB,"[Formal Garden, Mount Ascutney]",[https://www.nps.gov/media/webcam/view.htm?id=...
10AEFADA-AFA0-4998-A9E5-3F504F351AAB,[Visitor Center],[https://www.nps.gov/media/webcam/view.htm?id=...
11E73438-0CCC-4441-A76A-1995F67F2D89,[Bear Glacier Ice-Dammed Lake Webcam],[https://www.nps.gov/media/webcam/view.htm?id=...
1ABD0EFF-AC09-4EA1-8CC1-2351A3E160D0,[Webcam at the Painted Desert Inn National His...,[https://www.nps.gov/media/webcam/view.htm?id=...
1F38885B-538E-499C-B5A6-30D76267AC7F,"[Florissant valley, Florissant Fossil Beds]",[https://www.nps.gov/media/webcam/view.htm?id=...


In [9]:
activities = pd.read_sql_query('select * from park_activities', con=engine)
activities = activities[['park_id', 'activity_id', 'activity_name']]
activities_df = pd.DataFrame(activities.groupby('park_id').agg({'activity_id': list,
                                                  'activity_name':list}))
activities_df

,activity_id,activity_name
park_id,,
00E65872-53F7-4C9B-BD29-B9BC99930D0D,"[5F723BAD-7359-48FC-98FA-631592256E35, 0B4A532...","[Auto and ATV, Scenic Driving, Camping, Group ..."
0103833B-8114-4483-9754-11B5171BCB20,"[09DF0950-D319-4557-A57E-04CD2F63FF42, C59E231...","[Arts and Culture, Craft Demonstrations, Cultu..."
021F297B-D672-4794-8C5E-BB58743F252A,"[7CE6E935-F839-4FEC-A63E-052B1DEF39D2, 071BA73...","[Biking, Boating, Fishing, Hiking, Paddling, C..."
0260D050-761A-4446-AB05-641986F9FDFB,"[B33DC9B6-0B7D-4322-BAD7-A13A34C584A3, C7D5A14...","[Guided Tours, Self-Guided Tours - Auto, Museu..."
0265E5A4-576A-4626-882A-05971DEE9B76,"[B33DC9B6-0B7D-4322-BAD7-A13A34C584A3, BFF8C02...","[Guided Tours, Hiking, Front-Country Hiking, J..."
...,...,...
FF6B2CDF-10F5-4A7F-9437-74C1E7104C65,"[09DF0950-D319-4557-A57E-04CD2F63FF42, C59E231...","[Arts and Culture, Craft Demonstrations, Cultu..."
FF73E2AA-E274-44E1-A8F5-9DD998B0F579,"[09DF0950-D319-4557-A57E-04CD2F63FF42, FAED7F9...","[Arts and Culture, Cultural Demonstrations, Au..."
FFC9F9C4-D79D-4CA7-AB0F-7A2AD30061CD,"[13A57703-BB1A-41A2-94B8-53B692EB7238, D37A000...","[Astronomy, Stargazing, Biking, Mountain Bikin..."


In [12]:
park_fire = park.merge(fire_count_df, how='left', on='park_id')
park_f_a = park_fire.merge(activities_df, how='left', on='park_id')
park_f_a_s = park_f_a.merge(species, how='left', on='park_id')
park_f_a_s_i = park_f_a_s.merge(park_list_images_df, how='left', on='park_id')
pfasiu_df = park_f_a_s_i.merge(webcam_url_df, how='left', on='park_id')
pfasiu_df.fillna(value='Not Available', inplace=True)
pfasiu_df = pfasiu_df.rename(columns={'park_name':'Park Name',
                                    'park_code':'Park Code',
                                    'park_url':'Park Website',
                                    'image_url':'Park Images',
                                    'designation':'Park Designation',
                                    'states':'States',
                                    'latitude':'Latitude',
                                    'longitude':'Longitude',
                                    'longitude':'Longitude',
                                    'directions_url':'Park Directions',
                                    'fire_count': 'Fire Count',
                                    'Amphibian': 'Amphibian Count',
                                    'Bird': 'Bird Count',
                                    'Mammal': 'Mammal Count',
                                    'Reptile': 'Reptile Count',
                                    'webcam_title': 'Park Webcam Name',
                                    'park_webcam_url': 'Park Webcam URL',
                                      'activity_name':'Activities'})
pfasiu_df = pfasiu_df[['Park Name', 'Park Website', 'Park Code', 'Latitude', 
                     'Longitude', 'States', 'Park Directions', 'Park Designation', 'Activities',
                     'Fire Count', 'Amphibian Count', 'Bird Count', 'Mammal Count',
                     'Reptile Count', 'Park Images', 'Park Webcam Name', 'Park Webcam URL']]
pfasiu_df.head()

,Park Name,Park Website,Park Code,Latitude,Longitude,States,Park Directions,Park Designation,Activities,Fire Count,Amphibian Count,Bird Count,Mammal Count,Reptile Count,Park Images,Park Webcam Name,Park Webcam URL
0,Abraham Lincoln Birthplace National Historical...,https://www.nps.gov/abli/index.htm,abli,37.585866,-85.673305,KY,http://www.nps.gov/abli/planyourvisit/directio...,National Historical Park,"[Astronomy, Stargazing, Food, Picnicking, Juni...",Not Available,Not Available,Not Available,Not Available,Not Available,[https://www.nps.gov/common/uploads/structured...,Not Available,Not Available
1,Acadia National Park,https://www.nps.gov/acad/index.htm,acad,44.409286,-68.247501,ME,http://www.nps.gov/acad/planyourvisit/directio...,National Park,"[Arts and Culture, Cultural Demonstrations, As...",73.0,11.0,214.0,37.0,7.0,[https://www.nps.gov/common/uploads/structured...,"[View from McFarland Hill, Eye on Acadia, Nort...",[https://www.nps.gov/subjects/air/webcams.htm?...
2,Adams National Historical Park,https://www.nps.gov/adam/index.htm,adam,42.255396,-71.011604,MA,http://www.nps.gov/adam/planyourvisit/directio...,National Historical Park,"[Guided Tours, Living History, First Person In...",Not Available,Not Available,Not Available,Not Available,Not Available,[https://www.nps.gov/common/uploads/structured...,Not Available,Not Available
3,African American Civil War Memorial,https://www.nps.gov/afam/index.htm,afam,38.916600,-77.026000,DC,http://www.nps.gov/afam/planyourvisit/directio...,Not Available,"[Guided Tours, Self-Guided Tours - Walking]",Not Available,Not Available,Not Available,Not Available,Not Available,[https://www.nps.gov/common/uploads/structured...,Not Available,Not Available
4,African Burial Ground National Monument,https://www.nps.gov/afbg/index.htm,afbg,40.714527,-74.004474,NY,http://www.nps.gov/afbg/planyourvisit/directio...,National Monument,"[Arts and Culture, Guided Tours, Junior Ranger...",Not Available,Not Available,Not Available,Not Available,Not Available,[https://www.nps.gov/common/uploads/structured...,Not Available,Not Available


In [13]:
html = pfasiu_df.to_html()
with open("html.txt","w") as text_file:
    text_file.write(html)